In [8]:
import sys
!{sys.executable} -m pip install wrds

  Using cached wrds-3.3.0-py3-none-any.whl.metadata (5.7 kB)
Using cached wrds-3.3.0-py3-none-any.whl (13 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 9.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 19.7 MB/s eta 0:00:00


In [4]:
import sys
print(sys.executable)

D:\Anaconda3\python.exe


In [1]:
import wrds
import pandas as pd

# Connect to WRDS
db = wrds.Connection()

Enter your WRDS username [Yuriii]: yuriii915
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [13]:
db.list_tables(library='mfl')

['mflink1', 'mflink2', 'mflink3']

In [14]:
# Download monthly macro predictors from the Lettau-Ludvigson database
macro_vars = ['yyyymm', 'dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy', 'svar']
#df_macro = db.get_table(library='macro_finance', table='predictor')[macro_vars]

In [ ]:
# Convert 'yyyymm' to datetime (month-end)
df_macro['date'] = pd.to_datetime(df_macro['yyyymm'], format='%Y%m') + pd.offsets.MonthEnd(0)

# Extract 'year' and 'month' from the converted date
df_macro['year'] = df_macro['date'].dt.year
df_macro['month'] = df_macro['date'].dt.month

In [ ]:
# Filter for the date range: Jan 2000 to Jan 2024
df_macro = df_macro[(df_macro['date'] >= '2000-01-31') & (df_macro['date'] <= '2024-01-31')]

# Reorder columns
ordered_cols = ['date', 'year', 'month'] + [col for col in macro_vars if col != 'yyyymm']
df_macro = df_macro[ordered_cols]

In [ ]:
# Save to CSV in your project folder
df_macro.to_csv('data/macro/macro_monthly.csv', index=False)

print("Macro predictors saved to 'data/macro/macro_monthly.csv'")

In [18]:
import sys
!{sys.executable} -m pip install fredapi

  Using cached fredapi-0.5.2-py3-none-any.whl.metadata (5.0 kB)
Using cached fredapi-0.5.2-py3-none-any.whl (11 kB)


In [19]:
# Import necessary libraries
import pandas as pd
from fredapi import Fred
import os

# Initialize Fred client with your API key
fred = Fred(api_key='e4c223fe06769e014e4a2b0c0717f2b0')

# Define the series you want to download
series_dict = {
    'tbl': 'TB3MS',    # 3-Month Treasury Bill
    'gs10': 'GS10',    # 10-Year Treasury Constant Maturity Rate
    'baa': 'BAA',      # Moody's BAA Corporate Bond Yield
    'aaa': 'AAA'       # Moody's AAA Corporate Bond Yield
}

# Download data from FRED and store in a dictionary
macro_data = {}
for name, fred_code in series_dict.items():
    print(f"Downloading {name} from FRED ({fred_code})...")
    data = fred.get_series(fred_code)
    macro_data[name] = data

# Combine all series into a single DataFrame
df_macro = pd.DataFrame(macro_data)
df_macro.index = pd.to_datetime(df_macro.index)
df_macro = df_macro.resample('M').last()  # Ensure monthly frequency
df_macro = df_macro.dropna()

# Compute derived indicators
df_macro['tms'] = df_macro['gs10'] - df_macro['tbl']  # Term Spread
df_macro['dfy'] = df_macro['baa'] - df_macro['aaa']   # Default Spread

# Reset index and add yyyymm for later merging
df_macro = df_macro.reset_index().rename(columns={'index': 'date'})
df_macro['yyyymm'] = df_macro['date'].dt.year * 100 + df_macro['date'].dt.month

# Save the FRED-based macro dataset
output_path = './data/macro/fred_monthly.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_macro.to_csv(output_path, index=False)

print(f"FRED macro data (with tms/dfy) saved to: {output_path}")

FRED macro data (with tms/dfy) saved to: ./data/macro/fred_monthly.csv


C:\Users\Yuriii\AppData\Local\Temp\ipykernel_8616\2531269857.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_macro = df_macro.resample('M').last()  # Ensure monthly frequency


In [66]:
import pandas as pd
import numpy as np

# Load Goyal raw data
df = pd.read_csv("../data/raw/PredictorData2024.xlsx - Monthly.csv")

# Filter by year range: 2000.01 to 2024.01
df = df.sort_values('yyyymm').reset_index(drop=True)
df = df[(df['yyyymm'] >= 200001) & (df['yyyymm'] <= 202401)]

In [67]:
# Convert necessary fields to numeric
cols_to_convert = ['D12', 'E12', 'Index', 'b/m', 'ntis', 'svar', 'BAA', 'AAA', 'lty', 'tbl']
for col in cols_to_convert:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False) \
                                 .str.replace('$', '', regex=False) \
                                 .str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [68]:
# Drop rows with missing values
#df = df.dropna(subset=cols_to_convert)

# Compute log-ratio variables
df['dp'] = np.log(df['D12']) - np.log(df['Index'])
df['ep'] = np.log(df['E12']) - np.log(df['Index'])

# Compute spreads
df['dfy'] = df['BAA'] - df['AAA']
df['tms'] = df['lty'] - df['tbl']

# Rename for consistency
df.rename(columns={'b/m': 'bm'}, inplace=True)

# Final selection
final_cols = ['yyyymm', 'dp', 'ep', 'bm', 'ntis', 'svar', 'dfy', 'tms', 'tbl']
df_macro = df[final_cols]

# Save
output_path = "D:/Projects/Portfolio/Portfolio_Investment_Strategy/data/macro/macro_monthly.csv"
df_macro.to_csv(output_path, index=False)

In [63]:
print(df['yyyymm'].min(), df['yyyymm'].max())

200001 202401


In [65]:
df_macro.head(2)

,yyyymm,dp,ep,bm,ntis,svar,dfy,tms,tbl
1548,200001,-4.423938,-3.346471,0.154654,0.025359,0.005206,0.0055,0.0134,0.0532
1549,200002,-4.402226,-3.307461,0.167056,0.027419,0.003000,0.0061,0.0091,0.0555
